In [1]:
import sys
sys.path.insert(0, "/u/home/m/michaelw/project-ngarud/microbiome_evolution/microbiome_evolution_MOUSE/")

import sample_utils
import config
import parse_midas_data
import os.path
import pylab
import sys
import numpy

import diversity_utils
import gene_diversity_utils

import stats_utils
from math import log10,ceil
from numpy.random import randint

import core_gene_utils
import gzip

In [2]:
private_snv_directory = '%sprivate_snvs/' % (parse_midas_data.data_directory)
intermediate_filename_template = '%s%s.txt.gz'  

min_coverage = config.min_median_coverage
min_sample_size = 6                                         #MW: changed this from 5 to 6, as that's the minimum number of 
                                                            #samples before the SNV is in a new host. However, we might 
                                                            #want to consider reducing this.
allowed_variant_types = set(['1D','2D','3D','4D'])


In [3]:
def load_private_snv_map(species_name):
# This definition is called whenever another script downstream uses the output of this data.

    intermediate_filename = intermediate_filename_template % (private_snv_directory, species_name)

    private_snv_map = {}

    if not os.path.isfile(intermediate_filename):
        return private_snv_map
    
    file = gzip.open(intermediate_filename,"r")
    file.readline() # header
    for line in file:
     
        items = line.split(",")
        
        contig = items[0].strip()
        location = long(items[1])
        gene_name = items[2].strip()
        variant_type = items[3].strip()
        host = items[4].strip()
        
        private_snv_map[(contig, location)] = (gene_name, variant_type, host)
        
    return private_snv_map


In [4]:
debug = False
chunk_size = 1000000000
species_name="Akkermansia_muciniphila_55290"
good_species_list = [species_name]

In [5]:
# Load subject and sample metadata
sys.stderr.write("Loading sample metadata...\n")
subject_sample_map = parse_midas_data.parse_subject_sample_map() #MW: changed this from sample_utils.parse_midas_data() to parse_midas_data.parse_subject_sample_map()
sys.stderr.write("Done!\n")

Loading sample metadata...
Done!


In [6]:
snp_samples = diversity_utils.calculate_haploid_samples(species_name, debug=debug)

In [7]:
sys.stderr.write("Proceeding with %d haploid samples!\n" % len(snp_samples))

Proceeding with 39 haploid samples!


In [8]:
core_genes = core_gene_utils.parse_core_genes(species_name)
non_shared_genes = core_gene_utils.parse_non_shared_reference_genes(species_name)
shared_pangenome_genes = core_gene_utils.parse_shared_genes(species_name)
sys.stderr.write("Done! %d core genes and %d shared genes and %d non-shared genes\n" % (len(core_genes), len(shared_pangenome_genes), len(non_shared_genes)))

Done! 1505 core genes and 476 shared genes and 2106 non-shared genes


In [9]:
private_snvs = []
        
final_line_number = 0

In [10]:
#This is in a while loop, but in this case, final_line_number < 1 after one iteration (so the loop ends)
dummy_samples, allele_counts_map, passed_sites_map, final_line_number = parse_midas_data.parse_snps(species_name, debug=debug, allowed_samples=snp_samples, chunk_size=chunk_size,initial_line_number=final_line_number, allowed_genes=core_genes)


1k sites processed...
2k sites processed...
3k sites processed...


In [12]:
snp_samples = dummy_samples

In [15]:
sys.stderr.write("Done! Loaded %d genes\n" % len(allele_counts_map.keys()))

Done! Loaded 1503 genes


In [29]:
allele_counts_map['349741.6.peg.1721']['1D']['alleles'][]

{'349741.6.peg.1723': {'1D': {'location': ('NC_010655', 1810066L),
   'sites': array([[ 987.,  970.,  970., ...,  945.,  967.,  971.],
          [ 970.,  988.,  981., ...,  944.,  961.,  981.],
          [ 970.,  981.,  993., ...,  944.,  968.,  983.],
          ...,
          [ 945.,  944.,  944., ...,  945.,  937.,  940.],
          [ 967.,  961.,  968., ...,  937.,  981.,  965.],
          [ 971.,  981.,  983., ...,  940.,  965., 1016.]])},
  '2D': {'location': ('NC_010655', 1810066L),
   'sites': array([[239., 237., 236., ..., 230., 237., 238.],
          [237., 237., 236., ..., 230., 235., 236.],
          [236., 236., 237., ..., 230., 236., 236.],
          ...,
          [230., 230., 230., ..., 230., 230., 229.],
          [237., 235., 236., ..., 230., 238., 237.],
          [238., 236., 236., ..., 229., 237., 248.]])},
  '3D': {'location': ('NC_010655', 1810066L),
   'sites': array([[33., 33., 32., ..., 32., 31., 33.],
          [33., 35., 34., ..., 32., 33., 35.],
          [3

In [41]:
chunk_private_snvs = diversity_utils.calculate_private_snvs(snp_samples, allele_counts_map, passed_sites_map)

In [43]:
private_snvs.extend(chunk_private_snvs)

In [45]:
if len(private_snvs)>0:
            
    intermediate_filename = intermediate_filename_template % (private_snv_directory, species_name)

    # Now add records
    output_file = gzip.open(intermediate_filename,"w")
    # Header
    output_file.write("contig, location, gene_name, var_type, host_id\n")
    for contig, location, gene_name, variant_type, host in private_snvs:

        record_str_items = [contig, str(location), gene_name, variant_type, host]           
        record_str = ", ".join(record_str_items)
        output_file.write(record_str)
        output_file.write("\n")

    output_file.close()

In [50]:
intermediate_filename

NameError: name 'intermediate_filename' is not defined

In [46]:
intermediate_filename

NameError: name 'intermediate_filename' is not defined